In [12]:
import cv2
import os
from PIL import Image
import numpy as np
import pickle

In [13]:
BASE_DIR = os.getcwd()
image_dir = os.path.join(BASE_DIR, 'images')

In [22]:
face_cascade = cv2.CascadeClassifier('files/haarcascade_frontalface_alt2.xml')
eye_cascade = cv2.CascadeClassifier('files/haarcascade_eye.xml')

In [14]:
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [15]:
current_id=0
label_ids = {}
x_train = []
y_labels = []

In [19]:
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('.png') or file.endswith('.JPG') or file.endswith('.jpg'):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ", "-").lower()
            if not  label in label_ids:
                label_ids[label]= current_id
                current_id+=1
                id_ = label_ids[label]
                print(label_ids)
            pil_image = Image.open(path).convert('L') #grayscale
            size = (512, 512)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            image_aray = np.array(pil_image, 'uint8')
            faces = face_cascade.detectMultiScale(image_aray, scaleFactor=1.5, minNeighbors=5)
            for (x,y,w,h) in faces:
                roi = image_aray[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)

In [20]:
with open('labels.pickle','wb') as f:
    pickle.dump(label_ids, f)

In [21]:
recognizer.train(x_train, np.array(y_labels))
recognizer.save('trainer.yml')